In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" # ferramenta para treinamento mais rápido de modelos de linguagem
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes # ferramentas e bibliotecas para otimização do treinamento de modelos, incluindo a otimização da memória e eficiência computacional
!pip install transformers datasets # bibliotecas da Hugging Face que facilitam o uso de modelos de aprendizado de máquina e o carregamento de datasets
!pip install triton # install the missing triton package
from unsloth import FastLanguageModel, is_bfloat16_supported # carregar e treinar rapidamente modelos de linguagem
import torch # biblioteca PyTorch para computação de tensor
import json
from datasets import load_dataset # função para carregar datasets de forma simplificada
from trl import SFTTrainer # classe do trl para treinar modelos de linguagem usando SFT (Supervised Fine-Tuning)
from transformers import TrainingArguments # classe da biblioteca transformers para definir os argumentos de treinamento

fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
] # lista de modelos que usam representação em 4 bits para eficiência de armazenamento e processamento. Esses modelos são versões otimizadas que consomem menos memória


# Carrega o modelo pré-treinado LLaMA em 4 bits
model1, tokenizer1 = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Prepara o modelo para inferência
model1 = FastLanguageModel.for_inference(model1)

def generate_response1(question1):
    inputs1 = tokenizer1(question1, return_tensors="pt") # Tokeniza a entrada
    outputs1 = model1.generate(**inputs1, max_new_tokens=64)
    return tokenizer1.decode(outputs1[0], skip_special_tokens=True)

# Fazendo as perguntas
questions1 = [
    "Please provide a product review for the Ballet Tutu.<|end_of_text|>",
    "What materials are used to make a ballet tutu?<|end_of_text|>",
    "What sizes are available for the ballet tutu?<|end_of_text|>"
]

# Gerando e imprimindo as respostas
for question1 in questions1:
    print(generate_response1(question1))

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-t985za09/unsloth_b1da5a6fe48647ebaa4d72d05e5fa0bc
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-t985za09/unsloth_b1da5a6fe48647ebaa4d72d05e5fa0bc
  Resolved https://github.com/unslothai/unsloth.git to commit ae9e264e33c69b53dd5d533a4c5a264af4141c28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Please provide a

In [2]:
import json
import html

# Carregar o arquivo JSON linha por linha
file_path = '/content/drive/MyDrive/LLM/trn.json'

data = []

with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

pre_processing_data = []

for item in data:
  if item.get('content'):
    title = item.get("title", "")
    content = html.unescape(item.get("content", ""))
    pre_processing_data.append({"input": title, "output": content})

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/LLM/formatted_pre_processed_train_data.json"

# Inicializando as listas para armazenar os dados
instructions = []
inputs = []
outputs = []

# Processando o dataset
for prompt in pre_processing_data:
    instructions.append("give a review for the product")
    inputs.append(prompt.get("input", ""))
    outputs.append(prompt.get("output", ""))

# Criando o dicionário final
formatted_data = {
    "instruction": instructions,
    "input": inputs,
    "output": outputs
}

# Salvando o resultado em um arquivo JSON
with open(OUTPUT_PATH_DATASET, 'w') as output_file:
    json.dump(formatted_data, output_file, indent=4)

print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = "<|end_of_text|>" # fim de sentença, é útil para indicar onde termina a resposta gerada pelo modelo

# recebe um conjunto de exemplos (instruções, entradas e saídas) e formata cada exemplo usando o template definido anteriormente
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

PATH_DATASET = "/content/drive/MyDrive/LLM/formatted_pre_processed_train_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-lexswr_n/unsloth_911a42689cda4536a2163289f031f5b0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-lexswr_n/unsloth_911a42689cda4536a2163289f031f5b0
  Resolved https://github.com/unslothai/unsloth.git to commit ae9e264e33c69b53dd5d533a4c5a264af4141c28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1498718 [00:00<?, ? examples/s]

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# configura um treinador (SFTTrainer) para ajustar um modelo de linguagem com um conjunto de dados. Ele define parâmetros de treinamento, como tamanho do lote, taxa de aprendizado,
# passos de aquecimento e tipo de otimizador, que são essenciais para otimizar o desempenho do modelo e garantir eficiência no uso da memória
trainer = SFTTrainer(
    model = model, # modelo de linguagem que você carregou e ajustou previamente
    tokenizer = tokenizer, #tokenizer associado ao modelo, que é usado para codificar e decodificar texto
    train_dataset = dataset, #conjunto de dados de treinamento
    dataset_text_field = "text", # campo no dataset que contém o texto formatado para treinamento
    max_seq_length = 2048, # comprimento máximo das sequências de entrada que o modelo irá aceitar
    dataset_num_proc = 2, # número de processos a serem usados ao carregar e processar o dataset. Um valor de 2 pode acelerar o processamento em máquinas com múltiplos núcleo
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2, #  tamanho do lote por dispositivo (neste caso, 2). Isso determina quantos exemplos serão processados por vez em cada iteração
        gradient_accumulation_steps = 4,
        warmup_steps = 5, # número de passos para o aquecimento do aprendizado
        max_steps = 60, # número total de passos de treinamento
        learning_rate = 2e-4, # taxa de aprendizado
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", #usar menos memoria
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Map (num_proc=2):   0%|          | 0/1498718 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,498,718 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.905800
2,3.047300
3,3.062100
4,2.963100
5,2.545800
6,2.646300
7,2.212500
8,2.091200
9,2.142600
10,1.841300


In [4]:
import json
news_test = []

model.save_pretrained("/content/drive/MyDrive/LLM/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/LLM/lora_model")

from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/LLM/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [13]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt") # Tokeniza a entrada
    outputs = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

questions = [
    "Please provide a product review for the Ballet Tutu.",
    "What materials are used to make a ballet tutu?",
    "What sizes are available for the ballet tutu?"
]

# Gerando e imprimindo as respostas
for question in questions:
    print(generate_response(question))

Please provide a product review for the Ballet Tutu. We would love to hear your feedback on this product.
The ballet tutu is a beautiful and colorful costume. It is a great costume for the ballet recital.
What materials are used to make a ballet tutu? How is it made?
A ballet tutu is made of a soft, thin, stretchy material. It is made by sewing many layers of the material together, and then gathering it into a cone shape.
What is the ballet tutu made of?
A ballet tutu is made of layers of tulle, a
What sizes are available for the ballet tutu? What color is available for the ballet tutu?
The ballet tutu is available in size M, L and XL. The color is black.
